In [1]:
# Import libraries
import pandas as pd
import numpy as np

In [2]:
# Load Data
data = pd.read_csv("processed_data.txt", sep="|")
data.head()

,Gender_Mapped,Age_Scaled,Sqrt_Salary_Scaled,Purchased
0,1,-1.781797,-1.785227,0
1,1,-0.253587,-1.730989,0
2,0,-1.113206,-0.732165,0
3,0,-1.017692,-0.256825,0
4,1,-1.781797,0.302609,0


In [3]:
# Define features
X = data[["Gender_Mapped", "Age_Scaled", "Sqrt_Salary_Scaled"]]

In [4]:
# Define target
y = data["Purchased"]

In [5]:
# Divide data into training and testing.
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=35)

In [6]:
# Convert train and test data sets into tensors
import tensorflow as tf
tf.random.set_seed(35)

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

In [7]:
# Define model and layers
model = tf.keras.Sequential([
    tf.keras.layers.Dense(len(X), activation="relu"), 
    tf.keras.layers.Dense(1, activation="sigmoid")
])

In [8]:
# Define model attributes
model.compile(
    optimizer=tf.keras.optimizers.Adam(), 
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=["accuracy"]
)

In [9]:
# Train model
model.fit(X_train, y_train, batch_size=5, epochs=50)

Epoch 1/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6796 - loss: 0.6255
Epoch 2/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8333 - loss: 0.4257
Epoch 3/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8385 - loss: 0.3562
Epoch 4/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8655 - loss: 0.3201
Epoch 5/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8877 - loss: 0.2963
Epoch 6/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8882 - loss: 0.2794
Epoch 7/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8981 - loss: 0.2674
Epoch 8/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8981 - loss: 0.2589
Epoch 9/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9034 - loss: 0.2527
Epoch 10/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9099 - loss: 0.2479
Epoch 11/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9099 - loss: 0.2442
Epoch 12/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9099 - lo

In [10]:
# Evaluate model
results = model.evaluate(X_test, y_test, batch_size=5)
print(results)

20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9398 - loss: 0.2127
[0.265577495098114, 0.9300000071525574]


In [11]:
# Confusion Matrix
y_pred = model.predict(X_test)

tf.math.confusion_matrix(y_test, y_pred)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step


<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[64,  0],
       [36,  0]])>

In [12]:
# Import scaler object from "Preprocessing" notebook
%store -r scaler

In [13]:
# New data for model to make predictions. [Gender, Age, SqRt. of Salary]
new_data = [
[1, 20, 450],
[0, 64, 150],
[1, 32, 370],
[0, 30, 200],
[0, 28, 100],
[1, 70, 350],
[1, 61, 275],
[1, 42, 275],
[0, 56, 275],
[1, 47, 375]
]

In [14]:
# Function to make predictions from a set of input tensors
def predict_with_new_data(input):
    scaled_input = scaler.fit_transform(input)
    scaled_input_tensor = tf.convert_to_tensor(scaled_input, dtype=tf.float32)
    prediction = model.predict(tf.stack(scaled_input_tensor))
    return tf.round(prediction * 1000.0) / 1000.0

In [15]:
predict_with_new_data(new_data)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step


<tf.Tensor: shape=(10, 1), dtype=float32, numpy=
array([[0.187],
       [0.324],
       [0.113],
       [0.   ],
       [0.   ],
       [0.917],
       [0.428],
       [0.013],
       [0.012],
       [0.799]], dtype=float32)>